**Objective: for all the models created, test result and combine models for majority votting. Final F1 score is 0.99**

## Mount google drive to colab

In [12]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set to GPU

In [13]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')

print(physical_devices)

if len(physical_devices) >= 1:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

[]


## load test images

In [14]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [15]:
from os import listdir

## get test image names and ground truth labels

test_image_name_list = []

test_image_label_list = []

test_image_dir = "/content/drive/MyDrive/HPE_Proj/food_recognition_proj/images/test"
## note: if running in local, use the path below:
## test_image_dir = "images/test"

category_list = ['hamburger', 'pizza', 'ramen', 'sushi']

for category in category_list:
    image_fold = test_image_dir + "/" + category
    files = listdir(image_fold)

    test_image_name_list = test_image_name_list + files
    test_image_label_list = test_image_label_list + [category] * len(files)

    print("there are {} test images with label {}".format(len(files), category))

there are 150 test images with label hamburger
there are 150 test images with label pizza
there are 150 test images with label ramen
there are 150 test images with label sushi


## define model prediction function

In [16]:
image_size = (256, 256, 3)

def predictResult(input_model, input_image_path):
    '''
    perform prediction, get predict category and probability
    '''
    image = keras.preprocessing.image.load_img(path=input_image_path, target_size = image_size )
    image_tensor = keras.preprocessing.image.img_to_array(image)
    predictProbabilities = input_model(image_tensor, training=False).numpy()

    #print(predictProbabilities)
    predict_index = predictProbabilities[0].argmax(axis=-1)
    #print("predict index is ", predict_index)
    predict_category = category_list[predict_index]
    probability = predictProbabilities[0][predict_index]

    return predict_category, probability
    

## Model 1: ResNet. 

In [17]:
resnet_model_path = "/content/drive/MyDrive/HPE_Proj/food_recognition_proj/ResNet_model_03"
## if run in local, use the path below:
## resnet_model_path = "ResNet_model_03"

model_resnet = keras.models.load_model(resnet_model_path)
model_resnet.summary()


Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
normalization_6 (Normalizati (None, 256, 256, 3)       7         
_________________________________________________________________
resnet152v2 (Functional)     (None, 8, 8, 2048)        58331648  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 2048)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)               

In [18]:
%%time
predictResult_resnet = []

predictProbability_resnet = []

for i in range(len(test_image_name_list)):
    image_path = test_image_dir + "/" + test_image_label_list[i] + "/" + test_image_name_list[i]  # get image path
    predict_category, predict_probability = predictResult(model_resnet, image_path)               # get predict result and probability
    #print("input image is ", image_path)
    #print("Predict result is {}, the probability is {}".format(predict_category, predict_probability) )

    predictResult_resnet.append(predict_category)
    predictProbability_resnet.append(predict_probability)


CPU times: user 12min 25s, sys: 18.2 s, total: 12min 43s
Wall time: 7min 27s


In [19]:
if len(predictResult_resnet) != len(test_image_name_list):
    raise Exception("Error, some images are missed")

print("number of test images is {}".format(len(predictResult_resnet)))

number of test images is 600


### generate classification report for ResNet Model

In [20]:
from sklearn.metrics import classification_report

## based on ground truth labels and prediction result, get the classification report
print(classification_report(test_image_label_list, predictResult_resnet))

              precision    recall  f1-score   support

   hamburger       0.99      0.99      0.99       150
       pizza       0.99      0.94      0.97       150
       ramen       0.95      1.00      0.97       150
       sushi       0.99      0.98      0.98       150

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [21]:
print("As shown in the classification report above, the weighted F1-score for ResNet model is 0.98")

As shown in the classification report above, the weighted F1-score for ResNet model is 0.98


## Model 2: Inception

In [22]:
inception_model_path = "/content/drive/MyDrive/HPE_Proj/food_recognition_proj/Inception_model_03"
# inception_model_path = "Inception_model_03"

model_inception = keras.models.load_model(inception_model_path)
model_inception.summary()


Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
normalization_8 (Normalizati (None, 256, 256, 3)       7         
_________________________________________________________________
inception_v3 (Functional)    (None, 6, 6, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_8 ( (None, 2048)              0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)               

In [23]:
%%time
predictResult_inception = []

predictProbability_inception = []

for i in range(len(test_image_name_list)):
    image_path = test_image_dir + "/" + test_image_label_list[i] + "/" + test_image_name_list[i]  # get image path
    predict_category, predict_probability = predictResult(model_inception, image_path)               # get predict result and probability
    #print("input image is ", image_path)
    #print("Predict result is {}, the probability is {}".format(predict_category, predict_probability) )

    predictResult_inception.append(predict_category)
    predictProbability_inception.append(predict_probability)

CPU times: user 3min 42s, sys: 5.48 s, total: 3min 47s
Wall time: 2min 26s


In [24]:
if len(predictResult_inception) != len(test_image_name_list):
    raise Exception("Error, some images are missed")

print("number of test images is {}".format(len(predictResult_inception)))

number of test images is 600


### generate classification report for Inception model

In [25]:
from sklearn.metrics import classification_report

## based on ground truth labels and prediction result, get the classification report
print(classification_report(test_image_label_list, predictResult_inception))

              precision    recall  f1-score   support

   hamburger       0.96      0.99      0.97       150
       pizza       0.99      0.97      0.98       150
       ramen       0.97      1.00      0.99       150
       sushi       0.99      0.96      0.97       150

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [26]:
print("As shown in the classification report above, the weighted F1-score for Inception model is 0.98")

As shown in the classification report above, the weighted F1-score for Inception model is 0.98


## Model 3: Xception

In [27]:
xception_model_path = "/content/drive/MyDrive/HPE_Proj/food_recognition_proj/Xception_model_03"
# xception_model_path = "Xception_model_03"

model_xception = keras.models.load_model(xception_model_path)
model_xception.summary()

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
normalization_15 (Normalizat (None, 256, 256, 3)       7         
_________________________________________________________________
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d_15  (None, 2048)              0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 4)               

In [28]:
%%time
predictResult_xception = []

predictProbability_xception = []

for i in range(len(test_image_name_list)):
    image_path = test_image_dir + "/" + test_image_label_list[i] + "/" + test_image_name_list[i]  # get image path
    predict_category, predict_probability = predictResult(model_xception, image_path)               # get predict result and probability
    #print("input image is ", image_path)
    #print("Predict result is {}, the probability is {}".format(predict_category, predict_probability) )

    predictResult_xception.append(predict_category)
    predictProbability_xception.append(predict_probability)

CPU times: user 6min 19s, sys: 7.53 s, total: 6min 27s
Wall time: 3min 41s


### generate classification report for Xception model

In [29]:
from sklearn.metrics import classification_report

## based on ground truth labels and prediction result, get the classification report
print(classification_report(test_image_label_list, predictResult_xception))

              precision    recall  f1-score   support

   hamburger       0.99      0.98      0.99       150
       pizza       0.98      0.99      0.98       150
       ramen       0.97      0.99      0.98       150
       sushi       0.98      0.97      0.98       150

    accuracy                           0.98       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.98      0.98       600



In [30]:
print("As shown in the classification report above, the weighted F1-score for Xception model is 0.98")

As shown in the classification report above, the weighted F1-score for Xception model is 0.98


## Model 4: InceptionResNet

In [31]:
inceptionResNet_model_path = "/content/drive/MyDrive/HPE_Proj/food_recognition_proj/InceptionResNet_model_03"
# inceptionResNet_model_path = "InceptionResNet_model_03"

model_inceptionResNet = keras.models.load_model(inceptionResNet_model_path)
model_inceptionResNet.summary()

Model: "functional_57"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_58 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 256, 256, 3)       0         
_________________________________________________________________
normalization_28 (Normalizat (None, 256, 256, 3)       7         
_________________________________________________________________
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_28  (None, 1536)              0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 1536)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 4)               

In [32]:
%%time
predictResult_inceptionResNet = []

predictProbability_inceptionResNet = []

for i in range(len(test_image_name_list)):
    image_path = test_image_dir + "/" + test_image_label_list[i] + "/" + test_image_name_list[i]  # get image path
    predict_category, predict_probability = predictResult(model_inceptionResNet, image_path)               # get predict result and probability
    #print("input image is ", image_path)
    #print("Predict result is {}, the probability is {}".format(predict_category, predict_probability) )

    predictResult_inceptionResNet.append(predict_category)
    predictProbability_inceptionResNet.append(predict_probability)

CPU times: user 8min 38s, sys: 15.9 s, total: 8min 54s
Wall time: 5min 45s


### generate classification report for inceptionResNet model

In [33]:
from sklearn.metrics import classification_report

## based on ground truth labels and prediction result, get the classification report
print(classification_report(test_image_label_list, predictResult_inceptionResNet))

              precision    recall  f1-score   support

   hamburger       0.98      0.97      0.97       150
       pizza       1.00      0.97      0.98       150
       ramen       0.96      1.00      0.98       150
       sushi       0.97      0.97      0.97       150

    accuracy                           0.97       600
   macro avg       0.98      0.98      0.98       600
weighted avg       0.98      0.97      0.98       600



In [34]:
print("As shown in the classification report above, the weighted F1-score for InceptionResNet model is 0.98")

As shown in the classification report above, the weighted F1-score for InceptionResNet model is 0.98


## Combine the top 3 best models: ResNet, Inception and Xception to perform majority votting

In [37]:
## Majority votting from ResNet, Inception and Xception models

def processVottingData(categories, probabilities):
    counter = 0
    category = None
    for i in categories:
        curr_frequency = categories.count(i)
        if (curr_frequency) > counter:
            counter = curr_frequency
            category = i
    
    if counter == 1:
        # if the max counter is 1, then we select the category which has max prediction probabilities. 
        index = probabilities.index(max(probabilities))
        return categories[index]
    else:
        # else return the category with the most frequency
        return category

predictResult_votting = []

for i in range(len(test_image_name_list)):
    temp_category = []
    temp_probability = []
    ## add predictions from ResNet, Inception and Xception into temp_category
    temp_category.append(predictResult_resnet[i])
    temp_category.append(predictResult_inception[i])
    temp_category.append(predictResult_xception[i])

    ## add probabilities from ResNet, Inception and Xception into temp_probability
    temp_probability.append(predictProbability_resnet[i])
    temp_probability.append(predictProbability_inception[i])
    temp_probability.append(predictProbability_xception[i])

    ## get votting
    votting_category = processVottingData(temp_category, temp_probability)
    predictResult_votting.append(votting_category)
    


### get classification report for votting

In [38]:
## get the votting result
print(classification_report(test_image_label_list, predictResult_votting))

              precision    recall  f1-score   support

   hamburger       0.99      0.99      0.99       150
       pizza       1.00      0.98      0.99       150
       ramen       0.97      1.00      0.99       150
       sushi       0.99      0.99      0.99       150

    accuracy                           0.99       600
   macro avg       0.99      0.99      0.99       600
weighted avg       0.99      0.99      0.99       600



In [39]:
print("From majority votting, the weighted F1-score is 0.99")

From majority votting, the weighted F1-score is 0.99


## save all the predictions to csv file

In [42]:
import pandas as pd

result_dict = {"image name" : test_image_name_list,
               "ground truth": test_image_label_list,
               "ResNet pred": predictResult_resnet,
               "ResNet pred proba": predictProbability_resnet,
               "Inception pred": predictResult_inception,
               "Inception pred proba": predictProbability_inception,
               "Xception pred": predictResult_xception,
               "Xception pred proba": predictProbability_xception,
               "IceptionResNet pred": predictResult_inceptionResNet,
               "IceptionResNet pred proba": predictProbability_inceptionResNet,
               "votting based on ResNet, Inception, Xception": predictResult_votting
               }

df = pd.DataFrame(result_dict)

In [43]:
df.head()

,image name,ground truth,ResNet pred,ResNet pred proba,Inception pred,Inception pred proba,Xception pred,Xception pred proba,IceptionResNet pred,IceptionResNet pred proba,"votting based on ResNet, Inception, Xception"
0,1213037.jpg,hamburger,hamburger,1.000000,hamburger,0.999732,hamburger,0.999845,hamburger,0.999997,hamburger
1,1381751.jpg,hamburger,hamburger,0.999979,hamburger,0.994313,hamburger,0.984473,hamburger,0.995442,hamburger
2,1327199.jpg,hamburger,hamburger,1.000000,hamburger,0.999808,hamburger,0.999657,hamburger,0.996029,hamburger
3,1255149.jpg,hamburger,hamburger,1.000000,hamburger,0.999846,hamburger,0.999767,hamburger,0.999523,hamburger
4,139558.jpg,hamburger,hamburger,0.999967,hamburger,0.999885,hamburger,0.999895,hamburger,0.999951,hamburger


In [46]:
df.shape

(600, 11)

In [44]:
df.to_csv("/content/drive/MyDrive/HPE_Proj/food_recognition_proj/test_result.csv", index=False)

#df.to_csv("test_result.csv", index=False)

### ERROR analysis: find the one with wrong votting result

In [47]:
wrong_predition_df = df[df["ground truth"] != df["votting based on ResNet, Inception, Xception"]]

In [48]:
wrong_predition_df.shape

(6, 11)

In [51]:
wrong_predition_df.head(6)

,image name,ground truth,ResNet pred,ResNet pred proba,Inception pred,Inception pred proba,Xception pred,Xception pred proba,IceptionResNet pred,IceptionResNet pred proba,"votting based on ResNet, Inception, Xception"
137,446192.jpg,hamburger,hamburger,0.643747,ramen,0.592676,ramen,0.561570,hamburger,0.805756,ramen
168,1047561.jpg,pizza,ramen,0.909069,ramen,0.497029,ramen,0.630875,ramen,0.687036,ramen
193,2426686.jpg,pizza,sushi,0.569225,sushi,0.996575,pizza,0.973725,sushi,0.987825,sushi
194,2091857.jpg,pizza,ramen,0.824976,pizza,0.639736,ramen,0.809583,pizza,0.997031,ramen
516,194976.jpg,sushi,hamburger,0.951253,sushi,0.957728,hamburger,0.583182,ramen,0.498619,hamburger
598,918194.jpg,sushi,pizza,0.437342,ramen,0.714212,ramen,0.668742,hamburger,0.999843,ramen
